In [0]:
%sql

SHOW TABLES

database,tableName,isTemporary
default,all_employees,false


In [0]:
%sql

SHOW TABLES in hive_metastore.default;

database,tableName,isTemporary
default,employees,false
,_sqldf,true


In [0]:
%sql

SELECT * FROM loony_databricks_ws.default.all_employees

id,empname,phone,email,city
1,Michelle,555-345-2222,a@loonycorn.com,King of Prussia
2,Ben,555-231-4521,b@loonycorn.com,Bloomington
3,Kristoff,555-124-2402,c@loonycorn.com,Santa Fe
4,Fatima,555-250-2386,d@loonycorn.com,Dayton
5,Eduardo,555-345-2699,e@loonycorn.com,Biloxi


In [0]:
%sql

SELECT * FROM employees

org.apache.spark.sql.catalyst.ExtendedAnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `employees` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [employees], [], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.tableNotFound(package.scala:90)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$2(CheckAnalysis.scala:224)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$2$adapted(CheckAnalysis.scala:197)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:287)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:286)
	at 

In [0]:
%sql

SELECT * FROM hive_metastore.default.employees;

id,empname,phone,email,city
1,Michelle,555-345-2222,a@loonycorn.com,King of Prussia
2,Ben,555-231-4521,b@loonycorn.com,Bloomington
3,Kristoff,555-124-2402,c@loonycorn.com,Santa Fe
4,Fatima,555-250-2386,d@loonycorn.com,Dayton
5,Eduardo,555-345-2699,e@loonycorn.com,Biloxi


#### View the table details
Note the location on DBFS. 

In [0]:
%sql

DESCRIBE DETAIL hive_metastore.default.employees;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,770a90a8-ef34-4ede-a2d1-4eddc26bb710,hive_metastore.default.employees,Created by the file upload UI,dbfs:/user/hive/warehouse/employees,2024-10-28T04:02:01.411Z,2024-10-28T04:02:03Z,List(),List(),1,1460,Map(delta.enableDeletionVectors -> true),3,7,List(deletionVectors),"Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)"


#### Use dbutils to get details about the table directory on DBFS

#### All delta tables have: 
- A directory containing table data in the Parquet file format.
- A sub-directory /_delta_log that contains metadata about table versions in JSON and Parquet format.

In [0]:
dbutils.fs.ls('dbfs:/user/hive/warehouse/employees')

[FileInfo(path='dbfs:/user/hive/warehouse/employees/_delta_log/', name='_delta_log/', size=0, modificationTime=1730088121000),
 FileInfo(path='dbfs:/user/hive/warehouse/employees/part-00000-d99799b3-bd02-49fd-a776-05c03969617a.c000.snappy.parquet', name='part-00000-d99799b3-bd02-49fd-a776-05c03969617a.c000.snappy.parquet', size=1460, modificationTime=1730088123000)]

In [0]:
%fs ls dbfs:/user/hive/warehouse/employees

path,name,size,modificationTime
dbfs:/user/hive/warehouse/employees/_delta_log/,_delta_log/,0,1730088121000
dbfs:/user/hive/warehouse/employees/part-00000-d99799b3-bd02-49fd-a776-05c03969617a.c000.snappy.parquet,part-00000-d99799b3-bd02-49fd-a776-05c03969617a.c000.snappy.parquet,1460,1730088123000
dbfs:/user/hive/warehouse/employees/part-00000-e5d3b3de-5b70-4a6e-be57-57e17df5ae13.c000.snappy.parquet,part-00000-e5d3b3de-5b70-4a6e-be57-57e17df5ae13.c000.snappy.parquet,1208,1730088785000


#### Add rows with INSERT
Here, we don't explicitly specify the columns - just pass in the values in the correct order

In [0]:
%sql

INSERT INTO hive_metastore.default.employees 
VALUES (6, 'Amy', '555-001-6723', 'f@loonycorn.com', 'Spokane')

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql

SELECT * FROM hive_metastore.default.employees;

id,empname,phone,email,city
1,Michelle,555-345-2222,a@loonycorn.com,King of Prussia
2,Ben,555-231-4521,b@loonycorn.com,Bloomington
3,Kristoff,555-124-2402,c@loonycorn.com,Santa Fe
4,Fatima,555-250-2386,d@loonycorn.com,Dayton
5,Eduardo,555-345-2699,e@loonycorn.com,Biloxi
6,Amy,555-001-6723,f@loonycorn.com,Spokane


#### Check the underlying data for the table
It is likely a new .parquet file has been created. Delta maintains a version history, so it will be possible for us to roll back (time-travel in Delta-speak) to the previous version.

In [0]:
%fs ls dbfs:/user/hive/warehouse/employees

path,name,size,modificationTime
dbfs:/user/hive/warehouse/employees/_delta_log/,_delta_log/,0,1730088121000
dbfs:/user/hive/warehouse/employees/part-00000-d99799b3-bd02-49fd-a776-05c03969617a.c000.snappy.parquet,part-00000-d99799b3-bd02-49fd-a776-05c03969617a.c000.snappy.parquet,1460,1730088123000
dbfs:/user/hive/warehouse/employees/part-00000-e5d3b3de-5b70-4a6e-be57-57e17df5ae13.c000.snappy.parquet,part-00000-e5d3b3de-5b70-4a6e-be57-57e17df5ae13.c000.snappy.parquet,1208,1730088785000


#### View the version history of the Delta table

In [0]:
%sql

DESCRIBE HISTORY hive_metastore.default.employees;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2024-10-28T04:13:05Z,599420620358905,contact@loonycorn.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(3951670535128475),1028-032811-163l393x,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1208)",null,Databricks-Runtime/15.4.x-photon-scala2.12
0,2024-10-28T04:02:03Z,599420620358905,contact@loonycorn.com,CREATE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> Created by the file upload UI, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> false)",null,null,1028-032811-163l393x,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 5, numOutputBytes -> 1460)",null,Databricks-Runtime/15.4.x-photon-scala2.12


In [0]:
%sql

SELECT * FROM hive_metastore.default.employees;

id,empname,phone,email,city
1,Michelle,555-345-2222,a@loonycorn.com,King of Prussia
2,Ben,555-231-4521,b@loonycorn.com,Bloomington
3,Kristoff,555-124-2402,c@loonycorn.com,Santa Fe
4,Fatima,555-250-2386,d@loonycorn.com,Dayton
5,Eduardo,555-345-2699,e@loonycorn.com,Biloxi
6,Amy,555-001-6723,f@loonycorn.com,Spokane


In [0]:
_sqldf.display()

id,empname,phone,email,city
1,Michelle,555-345-2222,a@loonycorn.com,King of Prussia
2,Ben,555-231-4521,b@loonycorn.com,Bloomington
3,Kristoff,555-124-2402,c@loonycorn.com,Santa Fe
4,Fatima,555-250-2386,d@loonycorn.com,Dayton
5,Eduardo,555-345-2699,e@loonycorn.com,Biloxi
6,Amy,555-001-6723,f@loonycorn.com,Spokane
